In [2]:
### IMPORT STATEMENTS ###

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import string
from models import *
#from configs import cfg
from nltk.translate import bleu_score

In [3]:
### HELPER FUNCTIONS ###

# returns a pandas dataframe of the given file
def load_data(fname):
    return pd.read_csv(fname)


def process_train_data(data):
    # TODO: Input is a pandas DataFrame and return a numpy array (or a torch Tensor/ Variable)
    # that has all features (including characters in one hot encoded form).
    raise NotImplementedError

    
def train_valid_split(data, labels):
    # TODO: Takes in train data and labels as numpy array (or a torch Tensor/ Variable) and
    # splits it into training and validation data.
    raise NotImplementedError
    
    
def process_test_data(data):
    # TODO: Takes in pandas DataFrame and returns a numpy array (or a torch Tensor/ Variable)
    # that has all input features. Note that test data does not contain any review so you don't
    # have to worry about one hot encoding the data.
    raise NotImplementedError

    
def pad_data(orig_data):
    # TODO: Since you will be training in batches and training sample of each batch may have reviews
    # of varying lengths, you will need to pad your data so that all samples have reviews of length
    # equal to the longest review in a batch. You will pad all the sequences with <EOS> character 
    # representation in one hot encoding.
    raise NotImplementedError
    

def train(model, X_train, y_train, X_valid, y_valid, cfg):
    # TODO: Train the model!
    raise NotImplementedError
    
    
def generate(model, X_test, cfg):
    # TODO: Given n rows in test data, generate a list of n strings, where each string is the review
    # corresponding to each input row in test data.
    raise NotImplementedError
    
    
def save_to_file(outputs, fname):
    # TODO: Given the list of generated review outputs and output file name, save all these reviews to
    # the file in .txt format.
    raise NotImplementedError


In [4]:
train_data_fname = "Beeradvocate_Train_Short.csv"
test_data_fname = "Beeradvocate_Test_Short"
out_fname = "Output_Reviews.txt"

'''
train_data = load_data(train_data_fname) # Generating the pandas DataFrame
test_data = load_data(test_data_fname) # Generating the pandas DataFrame
train_data, train_labels = process_train_data(train_data) # Converting DataFrame to numpy array
X_train, y_train, X_valid, y_valid = train_valid_split(train_data, train_labels) # Splitting the train data into train-valid data
X_test = process_test_data(test_data) # Converting DataFrame to numpy array

model = baselineLSTM(cfg) # Replace this with model = <your model name>(cfg)
if cfg['cuda']:
    computing_device = torch.device("cuda")
else:
    computing_device = torch.device("cpu")
model.to(computing_device)

train(model, X_train, y_train, X_valid, y_valid, cfg) # Train the model
outputs = generate(model, X_test, cfg) # Generate the outputs for test data
save_to_file(outputs, out_fname) # Save the generated outputs to a file
'''

'\ntrain_data = load_data(train_data_fname) # Generating the pandas DataFrame\ntest_data = load_data(test_data_fname) # Generating the pandas DataFrame\ntrain_data, train_labels = process_train_data(train_data) # Converting DataFrame to numpy array\nX_train, y_train, X_valid, y_valid = train_valid_split(train_data, train_labels) # Splitting the train data into train-valid data\nX_test = process_test_data(test_data) # Converting DataFrame to numpy array\n\nmodel = baselineLSTM(cfg) # Replace this with model = <your model name>(cfg)\nif cfg[\'cuda\']:\n    computing_device = torch.device("cuda")\nelse:\n    computing_device = torch.device("cpu")\nmodel.to(computing_device)\n\ntrain(model, X_train, y_train, X_valid, y_valid, cfg) # Train the model\noutputs = generate(model, X_test, cfg) # Generate the outputs for test data\nsave_to_file(outputs, out_fname) # Save the generated outputs to a file\n'

In [74]:
# loads the train data
dataset = load_data("Beeradvocate_Train.csv")
print (dataset.shape)

# full data set has 1269291 reviews

(1269291, 14)


In [75]:
# selects the data that will be used
dataset = dataset[['beer/style', 'review/overall', 'review/text']]
print (dataset.shape)

(1269291, 3)


In [76]:
# drops the rows with missing data
dataset.replace('', np.nan, inplace = True)
dataset.dropna(inplace = True)
dataset.reset_index(drop = True, inplace = True)
print (dataset.shape)

# full data dropped 291 rows -- count = 1269000

(1269000, 3)


In [51]:
dataset['review/text']

0       A lot of foam. But a lot.\tIn the smell some b...
1       Dark red color, light beige foam, average.\tIn...
2       Almost totally black. Beige foam, quite compac...
3       Golden yellow color. White, compact foam, quit...
4       According to the website, the style for the Ca...
5       Poured from the bottle into a Chimay goblet.\t...
6       22 oz bottle from "Lifesource" Salem. $3.95 Ni...
7       Bottle says "Malt beverage brewed with Ginger ...
8       I'm not sure why I picked this up... I like gi...
9       Poured from a 22oz bomber into my Drie Fontein...
10      OK, so the only reason I bought this while sho...
11      Notes from 6/24\t\tA: Bright golden glowing be...
12      22 oz. bomber,\t\tA: Pours a clear yellow with...
13      Is it possible for a bottle to be a novelty pr...
14      Brown in color, somewhere between a porter and...
15      Caldera presents yet another circumstance wher...
16      More of a 'dry' than a lager, tasted at the 20...
17      Pours 

In [77]:
# cleans the data
dataset['beer/style'] = dataset['beer/style'].str.lower()
dataset['review/text'] = dataset['review/text'].str.lower()

char_list = string.ascii_lowercase + string.digits + '!$\'()*,-.:;? '

# converts all whitespace (space, tabs, newlines, etc.) into spaces
whitespace_regex = '[' + string.whitespace + ']'
dataset['review/text'] = dataset['review/text'].str.replace(whitespace_regex, ' ', regex = True)

# removes all invalid characters
invalid_char_regex = '[^' + char_list + ']'
dataset['review/text'] = dataset['review/text'].str.replace(invalid_char_regex, '', regex = True)

In [78]:
# creates lists of beer style and characters for a one-hot encoding
beer_list = dataset['beer/style'].unique()
beer_to_index = dict(zip(beer_list, range(beer_list.shape[0])))
char_to_index = dict(zip(list(char_list), range(len(char_list))))

print ("Number of unique beers: " + str(beer_list.shape[0]))

# full dataset: 104

Number of unique beers: 104


In [79]:
super_beer_list = beer_list
super_beer_to_index = beer_to_index

In [83]:
# converts the input features into a one-hot encoding
# result is a numpy array of features (one-hot) and a parallel numpy array of reviews (text)
num_data = dataset.shape[0]                   # number of data points in the dataset
input_feature_length = beer_list.shape[0] + 1 # one-hot encoding of beer style + rating

# converts the input features into a one-hot encoding
input_features = np.zeros((num_data, input_feature_length))
labels = dataset['review/text'].values

for index, review in dataset.iterrows():
    beer_style = review['beer/style']
    beer_index = beer_to_index[beer_style]
    score = review['review/overall']
    
    input_features[index][beer_index] = 1
    input_features[index][-1] = score

In [55]:
# splits the dataset into training and validation sets
percent_training = 0.8

training_last_index = int(percent_training * num_data)

x_train = input_features[:training_last_index]
y_train = labels[:training_last_index]

x_valid = input_features[training_last_index:]
y_valid = labels[training_last_index]

In [64]:
# loads the test data
dataset_test = load_data("Beeradvocate_Test.csv")
print (dataset_test.shape)

# full dataset: (317323, 13)

(317323, 13)


In [65]:
# selects the data that will be used
dataset_test = dataset_test[['beer/style', 'review/overall']]
print (dataset_test.shape)

(317323, 2)


In [68]:
# cleans the data
dataset_test['beer/style'] = dataset_test['beer/style'].str.lower()

In [84]:
# converts the input features into a one-hot encoding
# result is a numpy array of features (one-hot)
num_data_test = dataset_test.shape[0]                   # number of data points in the dataset

# converts the input features into a one-hot encoding
x_test = np.zeros((num_data_test, input_feature_length))

for index, review in dataset_test.iterrows():
    beer_style = review['beer/style']
    beer_index = beer_to_index[beer_style]
    score = review['review/overall']
    
    x_test[index][beer_index] = 1
    x_test[index][-1] = score

In [ ]:
# trains the model

train_loss = []
valid_loss = []

start = time.time()

if torch.cuda.is_available():
    computing_device = torch.device("cuda")
    
else:
    computing_device = torch.device("cpu")

model = bGRU(cfg).to(computing_device)
optimizer = optim.Adam(model.parameters(), cfg['learning_rate'])
criterion = nn.CrossEntropyLoss()

early_stop_count = 0
min_loss = 100
    
for epoch in range(cfg['epochs']):
    start_index = 0
    end_index = cfg['batch_size']
    
    losses = []
    
    while start_index < len(x_train):
        batch_x = x_train[start_index:end_index]
        batch_y = y_train[start_index:end_index]
        
        # convert the reviews char -> index
        indexed_reviews = []
        
        for review in batch_y:
            l = [char_to_index[x] for x in review]
            indexed_reviews.append(l)
        
        # pad the reviews
        max_review = 0
        for r in indexed_reviews:
            if len(r) > max_review:
                max_review = len(r)
        
        for i in len(indexed_reviews):
            to_pad = max_review - len(indexed_reviews[i])
            indexed_reviews[i] = [<SOS>] + indexed_reviews[i] + [<EOS>] + [<PAD>] * to_pad
        
        # concatenate reviews to input features        
        final_batch_x = []
        
        for j in range(cfg['batch_size']):
            review_list = []
            
            for i in range(max_review + 2):
                one_hot = [0] * (len(char_list) + 3)
                one_hot[indexed_review[i]] = 1
                i_f = batch_x[j] + one_hot
                review_list.append(i_f)
                
            final_batch_x.append(review_list)
        
        # converts input to tensors
        final_batch_x = torch.from_numpy(np.array(final_batch_x)).to(computing_device)
              
        optimizer.zero_grad()
        
        outputs = model(final_batch_x)
        
        # convert labels to digits
        
        loss = criterion(outputs, labels)
        
        losses.append(loss)
        
        loss.backward()
        
        optimizer.step()
        
    
    train_loss.append(np.mean(losses))
    
    # validate
    start_index = 0
    end_index = cfg['batch_size']
    
    losses = []
    
    with torch.no_grad():
        
        while start_index < len(x_valid):
            batch_x = x_valid[start_index:end_index]
            batch_y = y_valid[start_index:end_index]

            # one-hot encode the reviews char -> index -> one-hot

            # pad the reviews

            # concatenate reviews to input features

            # converts inputs (numpy arrays) to tensors
            batch_x = torch.from_numpy(batch_x).to(computing_device)
            batch_y = torch.from_numpy(batch_y).to(computing_device)


            outputs = model(batch_x)

            # convert labels to digits

            loss = criterion(outputs, labels)

            losses.append(loss)
            

    average_loss = np.mean(losses)
    valid_loss.append(average_loss)

    
    if average_loss >= min_loss:
        early_stop_count += 1

        if early_stop_count >= 2:
            break

    else:
        early_stop_count = 0
        min_loss = average_loss
